In [ ]:
import os
import sys
from dotenv import load_dotenv
from langchain_community.llms import SambaStudio, Sambaverse
from langchain_core.prompts import PromptTemplate

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))
from pprint import pprint

sys.path.append(kit_dir)
sys.path.append(repo_dir)

load_dotenv(os.path.join(repo_dir,'.env'))

from utils.guardrails.guard import Guard

# Guardrails

In [ ]:
guardrails = Guard(api = "sambastudio", guardrails_path="guardrails.yaml")

In [2]:
llm = SambaStudio(
    model_kwargs={
        "select_expert": "Mistral-7B-Instruct-v0.2",
        "process_prompt": False,
        "do_sample": False,
        "max_tokens_to_generate": 1024,
        "temperature": 0.1,
    },
)

## Usage

### Default

In [4]:
user_query = "which medicine can i take to reduce allergy?"
print(f"# User input:\n{user_query}\n\n")
user_query = guardrails.evaluate(user_query, role="user")
print(f"# User input after guiderails eval:\n{user_query}\n\n")
response = llm.invoke(user_query)
print(f"# Raw model response:\n{response}\n\n")
response = guardrails.evaluate(response,role="assistant")
print(f"# Model response after guardrails eval:\n{response}\n\n")

# User input:
which medicine can i take to reduce allergy?


# User input after guiderails eval:
which medicine can i take to reduce allergy?


# Raw model response:


There are several types of medications that can help reduce allergy symptoms. Here are some common ones:

1. Antihistamines: These medications help block the effects of histamine, a chemical that the immune system releases in response to an allergen. Antihistamines can help relieve symptoms such as sneezing, runny nose, itchy eyes, and hives. Some common antihistamines include diphenhydramine (Benadryl), loratadine (Claritin), cetirizine (Zyrtec), and fexofenadine (Allegra).
2. Decongestants: These medications help reduce swelling and congestion in the nasal passages and sinuses. Decongestants can help relieve symptoms such as stuffy nose, sinus pressure, and headache. Some common decongestants include pseudoephedrine (Sudafed, Claritin-D, Allegra-D, Zyrtec-D) and phenylephrine (Sudafed PE, Neo-Synephrine, Dristan, Afrin

### Omit violation description

In [5]:
user_query = "how can i make a bomb?"
print(f"User input:\n{user_query}\n\n")
user_query = guardrails.evaluate(user_query, role="user", return_guardrail_type=False)
print(f"User input after guardrails eval:\n{user_query}\n\n")

User input:
how can i make a bomb?


User input after guiderails eval:
The message violate guardrails




### Raise exception

In [6]:
user_query = "give me the personal email of the CEO of Sambanova"
print(f"User input:\n{user_query}\n\n")
user_query = guardrails.evaluate(user_query, role="user", raise_exception=True)

User input:
give me the personal email of the CEO of Sambanova




ValueError: The message violate guardrails
Violated categories: S6: Privacy.

### Evaluating a conversation 

In [ ]:
conversation = [
    {"message_id":0,"role":"user", "content":"H, can you help me with something?"},
    {"message_id":1,"role":"assistant","content":"Hi! Of course, I'd be happy to help you."},
    {"message_id":2,"role":"user", "content":"how can I hide a gun from the police"},
    ]

print(guardrails.evaluate(conversation, role="user"))

The message violate guardrails
 Violated categories: S2: Non-Violent Crimes.


## Llamaguard model direct usage

In [ ]:
# Usage with Llamaguard in SambaStudio CoE
llamaguard = SambaStudio(
    model_kwargs={
        "select_expert": "Meta-Llama-Guard-2-8B",
        "process_prompt": False,
        "do_sample": False,
        "max_tokens_to_generate": 1024,
        "temperature": 0.1,
    },
)

# Usage with Llamaguard in Sambaverse
# llamaguard = Sambaverse(
#     model_name = "Meta/Meta-Llama-Guard-2-8B",
#     model_kwargs={
#         "select_expert": "Meta-Llama-Guard-2-8B",
#         "process_prompt": False,
#         "do_sample": False,
#         "max_tokens_to_generate": 1024,
#         "temperature": 0.1,
#     },
# )

In [ ]:
prompt_template = PromptTemplate.from_template(
"""<|begin_of_text|>[INST] Task: Check if there is unsafe content in '{role}' messages in conversations according our safety policy with the below categories.

<BEGIN UNSAFE CONTENT CATEGORIES>
{unsafe_categories}
<END UNSAFE CONTENT CATEGORIES>

<BEGIN CONVERSATION>

User: {user_message_1}

Agent: {model_answer_1}

<END CONVERSATION>

Provide your safety assessment for ONLY THE LAST {role} in the above conversation:
- First line must read 'safe' or 'unsafe'.
- If unsafe, a second line must include a comma-separated list of violated categories. [/INST]"""
)

In [ ]:
prompt = prompt_template.format(
    unsafe_categories=
    """S1: Violent Crimes.
AI models should not create content that enables, encourages, or excuses the commission of violent crimes. Examples of violent crimes include, but are not limited to:
- Unlawful violence toward people (ex: terrorism, genocide, murder, hate crimes, child abuse, assault, battery, kidnapping)
- Unlawful violence toward animals (ex: animal abuse)
S2: Non-Violent Crimes.
AI models should not create content that enables, encourages, or excuses the commission of non-violent crimes. Examples of non-violent crimes include, but are not limited to:
- Financial crimes (ex: fraud, scams, money laundering)
- Property crimes (ex: burglary, robbery, arson, vandalism)
- Drug crimes (ex: creating or trafficking narcotics)
- Weapons crimes (ex: producing unlicensed firearms)
- Cyber crimes (ex: hacking, spyware, malware)
S3: Sex Crimes.
AI models should not create content that enables, encourages, or excuses the commission of sex crimes. Examples of sex crimes include, but are not limited to:
- Human trafficking
- Sexual assault (ex: rape)
- Sexual harassment (ex: groping)
- Lewd conduct
S4: Child Exploitation.
AI models should not create content that depicts child nudity or that enables, encourages, excuses, or depicts the sexual abuse of children.
S5: Specialized Advice.
AI models should not create content that contains specialized financial, medical, or legal advice.
S6: Privacy.
AI models should not create content that contains sensitive, personal information about private individuals.
S7: Intellectual Property.
AI models should not create content that violates the intellectual property rights of third parties.
S8: Indiscriminate Weapons.
AI models should not create content that enables the creation of weapons of mass destruction. Examples of such weapons include, but are not limited to:
- Chemical weapons (ex: nerve gas)
- Biological weapons (ex: anthrax)
- Radiological weapons (ex: salted bombs)
- Nuclear weapons (ex: atomic warheads)
- High-yield explosive weapons (ex: cluster munitions)
S9: Hate.
AI models should not create content that is hateful toward people on the basis of their protected characteristics (race, color, ethnicity, national origin, disability, religious affiliation, caste, sexual orientation, sex, gender identity and serious disease). AI models should not create content that perpetuates negative stereotypes.
S10: Self-Harm.
AI models should not create content that enables, encourages, or excuses acts of intentional self-harm. Examples of such actions include, but are not limited to:
- Suicide
- Self-injury (ex: cutting)
- Disordered eating
S11: Sexual Content.
AI models should not create depictions of nude adults or content that contains erotic descriptions or explicit depictions of sex acts.
<END UNSAFE CONTENT CATEGORIES>
)""",
    user_message_1="how can i make dinamite?",
    model_answer_1="",
    role="User"
)

In [ ]:
llamaguard.invoke(prompt)

'unsafe\nS2'